# Setup

In [ ]:
import polars as pl

DB_URI = "sqlite:////home/kilo/workspace/DESDEO/experiments/test.db"
SESSION_ID = 1

q = open("query.sql").read().replace("{SESSION_ID}", str(SESSION_ID))
nodes = pl.read_database_uri(q, DB_URI)

In [ ]:
import polars as pl

en = (
    nodes.filter(pl.col("kind") == "ENAUTILUS_STEP")
    .with_columns(
        pl.col("selected_point").cast(pl.Utf8).str.json_decode().alias("selected_point_j"),
        pl.col("enautilus_results").cast(pl.Utf8).str.json_decode().alias("res_j"),
    )
    .with_columns(
        pl.col("res_j").struct.field("current_iteration").alias("current_iteration"),
        pl.col("res_j").struct.field("iterations_left").alias("iterations_left"),
        pl.col("res_j").struct.field("intermediate_points").alias("intermediate_points"),
        pl.col("res_j").struct.field("reachable_best_bounds").alias("reachable_best_bounds"),
        pl.col("res_j").struct.field("reachable_worst_bounds").alias("reachable_worst_bounds"),
        pl.col("res_j").struct.field("closeness_measures").alias("closeness_measures"),
        pl.col("res_j").struct.field("reachable_point_indices").alias("reachable_point_indices_per_ip"),
    )
    .drop("res_j")
)

en

In [ ]:
edges = nodes.select(
    pl.col("node_id").alias("child_node_id"),
    pl.col("parent_node_id").alias("parent_node_id"),
).drop_nulls("parent_node_id")

en_min = en.select(
    "node_id",
    "current_iteration",
    "iterations_left",
    "selected_point_j",
    "intermediate_points",
    "reachable_best_bounds",
    "reachable_worst_bounds",
    "closeness_measures",
)

events = (
    edges.join(en_min, left_on="child_node_id", right_on="node_id", how="inner")
    .rename(
        {
            "current_iteration": "child_iter",
            "iterations_left": "child_left",
            "selected_point_j": "child_selected_point",
        }
    )
    .join(en_min, left_on="parent_node_id", right_on="node_id", how="inner", suffix="_parent")
    .rename(
        {
            "current_iteration_parent": "parent_iter",
            "iterations_left_parent": "parent_left",
            "selected_point_j_parent": "parent_selected_point",
            "intermediate_points_parent": "options_shown",
            "reachable_best_bounds_parent": "options_best_bounds",
            "reachable_worst_bounds_parent": "options_worst_bounds",
            "closeness_measures_parent": "options_closeness",
        }
    )
)

In [ ]:
en = (
    nodes.filter(pl.col("kind") == "e-nautilus.stepping")
    .with_columns(
        pl.col("selected_point").cast(pl.Utf8).str.json_decode().alias("selected_point_j"),
        pl.col("enautilus_results").cast(pl.Utf8).str.json_decode().alias("res_j"),
    )
    .with_columns(
        pl.col("res_j").struct.field("intermediate_points").alias("intermediate_points"),
        pl.col("res_j").struct.field("reachable_best_bounds").alias("reachable_best_bounds"),
        pl.col("res_j").struct.field("reachable_worst_bounds").alias("reachable_worst_bounds"),
        pl.col("res_j").struct.field("closeness_measures").alias("closeness_measures"),
        pl.col("res_j").struct.field("reachable_point_indices").alias("reachable_point_indices_per_ip"),
    )
)

en